# Crossword Local Search

## 概要
これはクロスワード(スケルトンパズル)自動生成用ソースをPythonにて再現できないかをテストしたものである。
main.cはpythonで代用し、計算量の多い処理は既存のcommon.cをimportして使用する。
結果やスコアの推移の視覚化も追加する。

***

## 入力データ・実行パラメータ設定
入力データを指定し、各種実行パラメータの設定を行います。
各パラメータは以下の通り：
  * `fpath`      : 入力データ(単語リスト)のファイルパス
  * `width`          : 盤面の大きさ(横)
  * `height`          : 盤面の大きさ(縦)
  * `randomSeed`       : シード値
  * `withweight` : 辞書に重みを付すかどうか(bool)
  * `takemove`   : "摂動(move)"を行うかどうか(bool)

In [55]:
fpath = "../dict/countries.txt" # countries.txt hokkaido.txt
width = 10
height = 5
randomSeed = 10
withweight = False
takemove = True

***

## Import
必要なライブラリをimportする：

In [56]:
import numpy as np
from numpy.random import *
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import unicodedata
import sys

from IPython.display import display
from PIL import Image
from IPython.display import HTML
%matplotlib inline

***

## クラス宣言
本プログラムで使用するクラスを定義する。
見やすさのため、クラスメソッドは後から定義し、`setattr`関数でアトリビュートに追加します。


### Puzzle クラス
解となるスケルトンパズルそのものを表すクラス。
メンバ変数は以下の通り：
  * width : 盤面の大きさ(横)
  * height : 盤面の大きさ(縦)
  * enable : 配置禁止マスを保持した2次元(width*height)配列
  * cell : 作業用2次元(width*height)配列
  * cover : セル上の文字数を保持する2次元(width*height)配列
  * sol : パズルの解を保存する2次元(width*height)配列
  * solSize : パズルに配置されている単語の数

In [57]:
class Puzzle():
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.enable = np.ones(width*height, dtype = "bool").reshape(height,width)
        self.cell = np.full(width*height, " ", dtype = "unicode").reshape(height,width)
        self.cover = np.zeros(width*height).reshape(height,width)
        self.sol = np.full(width*height, " ",  dtype = "unicode").reshape(height,width)
        self.solSize = 0
        ## Message
        print("Puzzle object has made.")
        print(" - width       : %d" % self.width)
        print(" - height      : %d" % self.height)
        print(" - cell' shape : (width, height) = (%d,%d)" % (self.cell.shape[0], self.cell.shape[1]))

In [58]:
puzzle = Puzzle(width, height)

Puzzle object has made.
 - width       : 10
 - height      : 5
 - cell' shape : (width, height) = (5,10)


### Dictionary クラス
入力した単語リストを整理して保持するクラス。
メンバ変数は以下の通り：
  * fpath : 入力データのファイルパス
  * size : 辞書の大きさ(単語数)
  * dictType : 辞書のタイプ("English"/"Japanese")
  * data : 入力データ配列

In [59]:
class Dictionary():        
    def __init__(self, fpath="unknown"):
        self.fpath = fpath
        print("Dictionary object has made.")
        
        ## Read
        print(" - READING DICTIONARY...")
        file = open(self.fpath)
        self.data = file.readlines()
        file.close()
        # Get a size of dictionary
        self.size = len(self.data)
        # Check dictionary type(English/Japanese)
        uniName = unicodedata.name(self.data[0][0])[0:-2]
        if "HIRAGANA" in uniName or "KATAKANA" in uniName:
            self.dictType = "Japanese"
        elif "LATIN CAPITAL" in uniName:
            self.dictType = "English"
            self.data = [s.upper() for s in self.data]
        # Remove "\n" & Check type for all letters
        for i in range(self.size):
            self.data[i] = self.data[i].rstrip("\n")
            for j in range(len(self.data[i])):
                if(unicodedata.name(self.data[i][j])[0:-2] != uniName):
                    sys.stderr.write("error: invalid English dictionary (%s)\n" % self.data[i])
                    sys.exit()
                    
        ## Message
        print(" - file path         : %s" % self.fpath)
        print(" - dictionary size   : %d" % self.size)
        print(" - dictionary type   : %s" % self.dictType)
        print(" - top of dictionary : %s" % self.data[0])

In [60]:
dic = Dictionary(fpath)

Dictionary object has made.
 - READING DICTIONARY...
 - file path         : ../dict/countries.txt
 - dictionary size   : 207
 - dictionary type   : English
 - top of dictionary : AFGHANISTAN


### Placeable クラス
辞書内のすべての単語に対して、それぞれの単語が配置可能(placeable)な位置の一覧を作る。これは`Puzzle`クラスと`Dictionary`クラスの両方の要素を用いて行われる。

配置可能な位置は、単語の先頭文字の座標で指定する。ここでは、パズルの左上を(0,0)、右上を(n,0)、左下を(0,n)、右下を(n,n)とする。
例えば、大きさが5×5のパズル上に`Dictionary`クラスの5番目に格納された長さ4文字の単語「HOGE」を配置しようとした場合、配置可能な位置は
  * 横向きの場合：(0,0),(0,1),(0,2),(0,3)(0,4),(1,0),(1,1),(1,2),(1,3),(1,4)の10マス。
  * 縦向きの場合：(0,0),(1,0),(2,0),(3,0)(4,0),(0,1),(1,1),(2,1),(3,1),(4,1)の10マス。
よって。配置する場合のパターンは全部で20通りになる。

In [61]:
#display(Image.open("fig/puzzles.png"))

これらの情報は次のフォーマットで整理される：
  * k : 単語番号(辞書内の何番目の単語か)
  * div : 単語を置く向き(0: 縦, 1: 横)
  * j : 単語の先頭文字のx座標
  * i : 単語の先頭文字のy座標

In [62]:
pd.DataFrame([[5,0,0,0],[5,0,0,1],[5,0,0,2],[5,0,0,3],[5,0,0,4],
              [5,0,1,0],[5,0,1,1],[5,0,1,2],[5,0,1,3],[5,0,1,4],
              [5,1,0,0],[5,1,1,0],[5,1,2,0],[5,1,3,0],[5,0,4,0],
              [5,1,0,1],[5,1,1,1],[5,1,2,1],[5,1,3,1],[5,1,4,1],],
             columns=["k","div","j","i"],
             index=["p1","p2","p3","p4","p5","p6","p7","p8","p9","p10",
                    "p11","p12","p13","p14","p15","p16","p17","p18","p19","p20"]
            )

,k,div,j,i
p1,5,0,0,0
p2,5,0,0,1
p3,5,0,0,2
p4,5,0,0,3
p5,5,0,0,4
p6,5,0,1,0
p7,5,0,1,1
p8,5,0,1,2
p9,5,0,1,3
p10,5,0,1,4


メンバ変数は以下の通り：
  * size : Placeableオブジェクトの大きさ
  * width : 引数のパズルの横幅
  * height : 引数のパズルの縦幅
  * dic : 引数の辞書
  * div : Placeable成分の文字列の方向
  * k : Placeable成分の文字の番号
  * i : Placeable成分のy方向の座標
  * j : Placeable成分のx方向の座標
  * used : Placeable成分が既に使われているかどうか(bool)
  * used_idx : 解として使われたPlaceable成分のインデックス集合
  * invP : Placeableオブジェクトの逆写像

In [63]:
class Placeable():
    def __init__(self, puzzle, dictionary):
        self.size = 0
        self.width = puzzle.width
        self.height = puzzle.height
        self.dic = dictionary
        self.div = np.zeros(2*self.dic.size*self.width*self.height, dtype='int64')
        self.k = np.zeros(2*self.dic.size*self.width*self.height, dtype='int64')
        self.i = np.zeros(2*self.dic.size*self.width*self.height, dtype='int64')
        self.j = np.zeros(2*self.dic.size*self.width*self.height, dtype='int64')
        self.used = np.zeros(2*self.dic.size*self.width*self.height, dtype = "bool")
        self.used_idx = np.zeros(self.width*self.height, dtype='int64')
        self.invP = np.zeros(2*self.dic.size*self.width*self.height, dtype='int64').reshape(2,self.dic.size,self.height,self.width)

        for div in range(2):
            for k in range(self.dic.size):
                if div == 0:
                    iMax = self.height - len(self.dic.data[k])
                    jMax = self.width - 1
                elif div == 1:
                    iMax = self.height - 1
                    jMax = self.width - len(self.dic.data[k])
                
                for i in range(iMax):
                    for j in range(jMax):
                        self.div[self.size] = div
                        self.k[self.size] = k
                        self.i[self.size] = i
                        self.j[self.size] = j
                        self.invP[div][k][i][j] = self.size
                        self.size += 1
        print("Placeable object has made.")
        print(" - placeable size : %d/%d(max shape)" % (self.size, self.div.size))

In [64]:
placeable = Placeable(puzzle, dic)

Placeable object has made.
 - placeable size : 2558/20700(max shape)


***
## パズル生成
ここから、実際にパズルを生成していく。
まずは、`placeable`オブジェクトからランダムに単語の配置可能位置を取得し、パズルの盤面に単語を詰められなくなるまで詰めます。
その準備として、`placeable`オブジェクトのインデックスの順番をランダムに取得できるよう、`randomIndex`を作成します：

In [65]:
# Make a random index of placeable
randomIndex = np.arange(placeable.size)
print(randomIndex)
seed(seed = randomSeed)
shuffle(randomIndex)
print(randomIndex)

[   0    1    2 ... 2555 2556 2557]
[1645  853 1277 ... 1344  527 1289]


次に、パズルに単語を詰めていきます。
単語を詰めるための関数として、`add`メソッドを`Puzzle`クラスに追加します。その前に、`add`メソッド内で呼ぶ、単語が配置可能かどうかをBoolianで返す`isEnabledAdd`メソッドを定義する：

In [66]:
### isEnabledAdd
def isEnabledAdd(self, plc, idx, wLen):    
    # If 0 words used, return True
    if self.solSize == 0:
        return True
    # If the word is already in use, return False
    if plc.used[idx] == True:
        return False
    # If the same word is in use, return False
    for used_idx in plc.used_idx:
        if plc.k[used_idx] == plc.k[idx]:
            return False
    # If the word does not fit in the puzzle, return False
    if plc.div[idx] == 0 and plc.i[idx]+wLen > self.height:
        return False
    if plc.div[idx] == 1 and plc.j[idx]+wLen > self.width:
        return False

    # Judge intersection and whether prohibited
    for p in range(wLen):
        crossing = False
        crossSame = False
        letter = plc.dic.data[plc.k[idx]][p]
        if plc.div[idx] == 0 and self.cell[plc.i[idx]+p][plc.j[idx]] != " ":
            crossing = True
            if self.enable[plc.i[idx]+p][plc.j[idx]] == True:
                crossSame = (letter == self.cell[plc.i[idx]+p][plc.j[idx]])
        elif plc.div[idx] == 1 and self.cell[plc.i[idx]][plc.j[idx]+p] != " ":
            crossing = True
            if self.enable[plc.i[idx]][plc.j[idx+p]] == True:
                crossSame = (letter == self.cell[plc.i[idx]][plc.j[idx]+p])
        if crossing == True and crossSame == False:
            return False

    # If neighbor cells are filled except at the intersection, return False
    for p in range(wLen):
        if plc.div[idx] == 0 and self.cell[plc.i[idx]+p][plc.j[idx]] == " ":
            # Left side
            if plc.j[idx] > 0 and self.cell[plc.i[idx]+p][plc.j[idx]-1] != " ":
                return False
            # Right side
            if plc.j[idx] < self.width-1 and self.cell[plc.i[idx]+p][plc.j[idx]+1] != " ":
                return False
        if plc.div[idx] == 1 and self.cell[plc.i[idx]][plc.j[idx]+p] == " ":
            # Upper
            if plc.i[idx] > 0 and self.cell[plc.i[idx]-1][plc.j[idx]+p] != " ":
                return False
            # Lower
            if plc.i[idx] < self.height-1 and self.cell[plc.i[idx]+1][plc.j[idx]+p] != " ":
                return False

    # If the preceding and succeeding cells are already filled
    if plc.div[idx] == 0:
        if plc.i[idx] > 0 and self.cell[plc.i[idx]-1][plc.j[idx]] != " ":
            return False
        if plc.i[idx]+wLen < self.height and self.cell[plc.i[idx]+wLen][plc.j[idx]] != " ":
            return False
    if plc.div[idx] == 1:
        if plc.j[idx] > 0 and self.cell[plc.i[idx]][plc.j[idx]-1] != " ":
            return False
        if plc.j[idx]+wLen < self.width and self.cell[plc.i[idx]][plc.j[idx]+wLen] != " ":
            return False
    
    # If Break through the all barrier, return True
    return True

# Set attribute to Puzzle class
setattr(Puzzle, "isEnabledAdd", isEnabledAdd)

次に、所望の単語をパズルに配置する`add`メソッドを定義する：

In [67]:
### add
def add(self, plc, idx):
    # Get the word length
    wLen = len(plc.dic.data[plc.k[idx]])
    # Judge whether adding is enabled
    if self.isEnabledAdd(plc, idx, wLen) == False:
        return
    
    # Put the word to puzzle
    if plc.div[idx] == 0:
        for p in range(wLen):
            self.cell[plc.i[idx]+p][plc.j[idx]] = plc.dic.data[plc.k[idx]][p]
    if plc.div[idx] == 1:
        for p in range(wLen):
            self.cell[plc.i[idx]][plc.j[idx]+p] = plc.dic.data[plc.k[idx]][p]

    # Set the prohibited cell before and after placed word
    if plc.div[idx] == 0:
        if plc.i[idx] > 0:
            self.enable[plc.i[idx]-1][plc.j[idx]] = False
        if plc.i[idx] < self.height:
            self.enable[plc.i[idx]+wLen][plc.j[idx]] = False
    if plc.div[idx] == 1:
        if plc.j[idx] > 0:
            self.enable[plc.i[idx]][plc.j[idx]-1] = False
        if plc.j[idx] < self.width:
            self.enable[plc.i[idx]][plc.j[idx]+wLen] = False
    
    # Update cover array
    if plc.div[idx] == 0:
        for p in range(wLen):
            self.cover[plc.i[idx]+p][plc.j[idx]] += 1
    if plc.div[idx] == 1:
        for p in range(wLen):
            self.cover[plc.i[idx]][plc.j[idx]+p] += 1
    
    # Update properties
    plc.used[idx] = True
    plc.used_idx[self.solSize] = idx
    self.solSize += 1
    return
# Set attribute to Puzzle class  
setattr(Puzzle, "add", add)

さあ、`add`メソッドが定義できたので、早速パズルを生成してみましょう。
`puzzle`オブジェクトで`add`メソッドを呼びます。
ただし、引数は`placeable`配列からランダムに与えます：

In [68]:
for t in range(placeable.size):
    puzzle.add(placeable, randomIndex[t])

結果が気になりますよね。
結果を確認するための`show`メソッドを定義します：

In [69]:
def show(self, ndarray):    
    styles = [
        dict(selector="th", props=[("font-size", "80%"),
                                   ("text-align", "center"),
                                   ("background", "#dddddd"),
                                   ("border", "solid 2px white"),
                                   ("width", "30px"),
                                   ("height", "30px")]),
        dict(selector="td", props=[("font-size", "105%"),
                                   ("text-align", "center"),
                                   ("background", "#dddddd"),
                                   ("border", "solid 2px white"),
                                   ("width", "30px"),
                                   ("height", "30px")]),
        dict(selector="caption", props=[("caption-side", "bottom")])
    ]

    df = pd.DataFrame(ndarray)
    df = (df.style.set_table_styles(styles)
          .set_caption("Puzzle(%d,%d), Seed:%d, solSize:%d Dictionary:[%s]" % (self.width, self.height, randomSeed, self.solSize, fpath)))
    return df
setattr(Puzzle, "show", show)

In [70]:
puzzle.show(puzzle.cover)

In [71]:
puzzle.show(puzzle.cell)